In [7]:
import csv
import numpy as np
from scipy.stats import rankdata
from pandas import read_csv, DataFrame
from itertools import product, chain, starmap, combinations, combinations_with_replacement
from time import time
# import fast
time0 = time()

k = 3
divisions = 1
range_ = 0.00
seed = 123

# 1. Function definitions

def discretize(seq, divisions=divisions, range_=range_, seed=seed):
    '''
    >>> discretize([3, 4, 1, 8, 13, 8], divisions=4, range_=0, seed=123) = array([1, 1, 0, 2, 3, 2])
    where
    ranks = [2., 3., 1., 4.5, 6., 4.5]
    tresholds = [1.5,  3.,  4.5]
    '''
    np.random.seed(seed)
    ranks = rankdata(seq, method='ordinal') # method='ordinal'/'average' ?

    random_blocks = np.cumsum(range_ * (2 * np.random.random(divisions + 1) - 1) + np.ones(divisions + 1))
    tresholds = random_blocks[:-1] / random_blocks[-1] * len(seq)
    
    discrete_seq = np.zeros(len(seq), dtype='float64')
    for treshold in tresholds:
        discrete_seq[ranks > treshold] += 1
    return discrete_seq

discretize_vec = np.vectorize(discretize, signature='(n)->(n)', excluded=['divisions', 'range_', 'seed'])

# 2. Read the data

file = "madelon_tiny.csv"
data = []
with open(file) as csvfile:
    reader = csv.reader(csvfile, delimiter=',',
                        quoting=csv.QUOTE_NONNUMERIC)
    for row in reader:
        data.append(row)
        
data = np.array(data, dtype='float64').T[:-1]
data[:-1] = discretize_vec(data[:-1])
data = data.astype('int64')

labels, counts = np.unique(data[-1], return_counts=True)
n_classes = len(labels)

xi = 1e-5
pseudo_counts = xi * counts / np.min(counts)


dim0, dim1 = data[:-1].shape

# 3. More function definitions

def tuple_generator(k=k, dim0=dim0):
    '''
    Python-generator.
    E.g. output for k=2:
    {0,1}, {0,2}, ..., {0, dim0-1}, {1,2}, ..., {1,dim0-1}, ..., {dim0-2, dim0-1}
    Go with combinations_with_replacement() to include diagonal tuples like {0, 0}
    '''        
    return combinations(range(dim0), k)    

def neg_H(p):
    return p * np.log2(p)

def neg_H_cond(matrix):
    return np.sum(neg_H(matrix)) - np.sum(neg_H(np.sum(matrix, axis=-1)))

def slow_work(indeces):
    '''
    indeces -> tuple
    Work-function.
    Output: tuple of Information Gains implicitly corresponding to the indeces
    '''
    # contingency-matrix: begin with pseudo-counts
    contingency_m = np.empty([divisions + 1] * k + [len(labels)], dtype='float64')
    for label, pseudo_count in enumerate(pseudo_counts):
        contingency_m[..., label] = pseudo_count
    
    # contingency-matrix: normal counts
    for c_index in data[list(indeces) + [-1]].T:
        contingency_m[tuple(c_index)] += 1
    
    results = []
    for i in range(len(indeces)):
        result = neg_H_cond(contingency_m) - neg_H_cond(np.sum(contingency_m, axis=i))
        results.append(result)
    return tuple(results)


def record(tuple_, IGs, records):
    '''
    tuple_, IGs -> tuple, tuple
    Accepts output of the work-function and updates the dict that accumulates global results
    '''
    for column, IG in zip(tuple_, IGs):
        if column not in records or IG > records[column][0]:
            records[column] = (IG, tuple_)

final_results = {}

for tuple_ in tuple_generator():
    IGs = slow_work(tuple_)
    #IGs = fast.work_2(dim1, divisions, data[tuple_[0]], data[tuple_[1]], n_classes, pseudo_counts, data[-1])
    #IGs = fast.work_3(dim1, divisions, data[tuple_[0]], data[tuple_[1]], data[tuple_[2]], n_classes, pseudo_counts, data[-1])
    record(tuple_, IGs, final_results)

# result
print("Finished in", time() - time0, "sec.")
result = DataFrame(final_results).T.rename(columns={0: 'IG_max', 1: 'tuple'})
result

Finished in 17.54623794555664 sec.


,IG_max,tuple
0,7.17026,"(0, 10, 19)"
1,12.762,"(1, 5, 12)"
2,46.7316,"(2, 5, 23)"
3,11.0879,"(3, 19, 25)"
4,6.8632,"(4, 24, 26)"
5,110.491,"(2, 5, 23)"
6,13.1498,"(6, 9, 26)"
7,11.9703,"(7, 19, 26)"
8,10.9056,"(8, 13, 29)"
9,9.88779,"(6, 9, 25)"
